# Caribbean seagrass

This is code for Southeast Conservation Blueprint 2023. Written by Rua Mordecai.

## Import libraries and define variables

In [81]:
import os
import arcpy

In [82]:
# define spatial reference, workspaces, and source data location
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\CaribbeanBlueprint2023_FinalIndicators\CaribbeanBlueprint2023_FinalIndicators\SpatialData"
benthicPR = r"D:\CaribbeanData\TNC\BenthicCondition\BENTHIC_CONDITION_PRVI\pr_mar_benhab4m_2021_EEZs_gridcode.tif"
benthicVI = r"D:\CaribbeanData\TNC\BenthicCondition\BENTHIC_CONDITION_PRVI\usvi_mar_benhab4m_2021_EEZs_gridcode.tif"
bathy = r"D:\CaribbeanData\BATHYMETRY\BATHYMETRY\PR_bathy_crm.tif"

In [83]:
# define final output name
IndicatorFileName = "CaribbeanSeagrass.tif"

In [84]:
# define raster used for cell size, extent, and snapping
CaribbeanRaster= r"D:\Blueprint\2023\extent\VIPR_Extent_v6.tif"

In [85]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

## Start analysis

### Prep data

In [86]:
# Set the source workspace
arcpy.env.workspace = OutputWorkspace

In [59]:
# Combine TNC benthic data
arcpy.management.MosaicToNewRaster(benthicPR+";"+benthicVI, OutputWorkspace, "vipr_mar_benhab4m_2021_EEZs_gridcode.tif", None, "8_BIT_UNSIGNED", None, 1, "LAST", "FIRST")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Seagrass\\working\\vipr_mar_benhab4m_2021_EEZs_gridcode.tif'>

In [60]:
# reproject benthic data and convert to 30 meters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
        arcpy.management.Resample("vipr_mar_benhab4m_2021_EEZs_gridcode.tif", "benthic.tif", "30 30", "MAJORITY")

In [61]:
# reclassify bathymetric data based on 13m and 5m shallow water threshold
out_raster = arcpy.sa.Reclassify(bathy, "Value", "-6488 -13 10;-13 -5 20;-5 0 30;0 1320 0", "DATA"); out_raster.save("bathyBin.tif")

In [62]:
# reproject bathymetryenthic data and convert to 30 meters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
        arcpy.management.Resample("bathyBin.tif", "bathyBin30.tif", "30 30", "MAJORITY")

### Reclassify benthic habitat and combine

In [63]:
# reclassify so dense seagrass is 2, sparse seagrass is 1, and everything else is 0
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify("benthic.tif", "Value", "1 11 0;12 1;13 2", "DATA"); out_raster.save("TNCSeagrass.tif")

In [64]:
# Combine rasters for indicator categories
out_raster = arcpy.sa.Plus("TNCSeagrass.tif", "bathyBin30.tif"); out_raster.save("SeagrassBathy.tif")

In [65]:
# Reclassify for indicator categories. Bathymetry data is a little coarse and miss some near shore areas
# so value of 1 and 2 are treated as shallow
out_raster = arcpy.sa.Reclassify("SeagrassBathy.tif", "Value", "0 0;1 5;2 6;10 0;11 1;12 2;20 0;21 3;22 4;30 0;31 5;32 6", "DATA"); out_raster.save("SeagrassBathyBin.tif")

In [66]:
# clip raster to Caribbean extent
out_raster = arcpy.sa.ExtractByMask("SeagrassBathyBin.tif", CaribbeanRaster); out_raster.save(IndicatorFileName)

## Finalize indicator

In [67]:
# set code block for legend
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = Not seagrass'    
    elif value == 1:
        return '1 = Deep and sparse seagrass'
    elif value == 2:
        return '2 = Deep and dense seagrass'
    elif value == 3:
        return '3 = Shallow and sparse seagrass'
    elif value == 4:
        return '4 = Shallow and dense seagrass'
    elif value == 5:
        return '5 = Very shallow and sparse seagrass'
    elif value == 6:
        return '6 = Very shallow and dense seagrass'
"""

In [68]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Seagrass\\working\\CaribbeanSeagrass.tif'>

In [87]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 6:
		return 7
	if Value == 5:
		return 18
	if Value == 4:
		return 44
	if Value == 3:
		return 88
	if Value == 2:
		return 130
	if Value == 1:
		return 211
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 6:
		return 64
	if Value == 5:
		return 93
	if Value == 4:
		return 130
	if Value == 3:
		return 169
	if Value == 2:
		return 208
	if Value == 1:
		return 242
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 6:
		return 80
	if Value == 5:
		return 104
	if Value == 4:
		return 123
	if Value == 3:
		return 133
	if Value == 2:
		return 144
	if Value == 1:
		return 163
	if Value == 0:
		return 255
	else:
		return 255
"""

In [88]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanSeagrass.tif'>